In [ ]:
#RouterChain
#RouterChain: Allows conditionally routing between different chains based on logic. Enables branching logic.

In [ ]:
#match teacher, spanish teacher, calculus teacher

In [1]:
math_teacher = """Your name is Helen, you are a Math teacher at  a\
primary school in Nairobi. You are very good at teaching math due to\
your ability to break-down complicated tasks into much smaller ones. \
Students ask different questions about math, you are responsible to answer them\
user your greate explanation skills to explain the concepts in very easy to understand manner.
All your responses should start in the format below:
Hello, am
          
Below is a question from a student in your 7th grade class:
{input}\
"""

spanish_teacher = """Your name is Thomas, you are a Spanish teacher at  a\
primary school in Nairobi. You are very good at teaching spanish due to\
your ability to explain spanish to non-spanish speaker in a fluent and easy to understand way. \
Students ask different questions about spanish, you are responsible to answer them\
user your greate explanation skills to explain the concepts in very easy to understand manner.
All your responses should start in the format below:
Hello, am
         
Below is a question from a student in your 7th grade class:
{input}\
"""


calculus_teacher = """Your name is Godfrey, you are a Calculus teacher at  a\
secondary(middle school) school in Nairobi. You are very good at teaching calculus due to\
your ability to explain complicated calculus topics using easy to understand real life examples. \
Students ask different questions about Caculus, you are responsible to answer them\
user your greate explanation skills to explain the concepts in very easy to understand manner.
All your responses should start in the format below:
Hello, am

Below is a question from a student in your 7th grade class:
{input}\
"""

In [2]:
prompt_infos = [
    {
        "name": "Math Teacher",
        "description": "Good for answering questions about Math",
        "prompt_template": math_teacher,
    },
    {
        "name": "Spanish Teacher",
        "description": "Good for answering questions about Spanish",
        "prompt_template": spanish_teacher,
    },
    {
        "name": "Calculus Teacher",
        "description": "Good for answering questions about Calculus",
        "prompt_template": calculus_teacher,
    },

]

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
llm = ChatGoogleGenerativeAI(model = "gemini-pro")

/Users/prasadk/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Build Destination Chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    template = p_info["prompt_template"]
    chat_prompt = ChatPromptTemplate.from_template(template=template)
    chain = LLMChain(llm=llm, prompt=chat_prompt)
    destination_chains[name]=chain

In [5]:
#Default Chain
default_prompt = ChatPromptTemplate.from_template(template='{input}')
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [6]:
from langchain.chains.router import MultiRouteChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [7]:
#Routing Destinations

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]

In [8]:
destination_str = '\n'.join(destinations)

In [9]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
router_prompt = PromptTemplate(template=router_template,
                              input_variables=["input"],
                              output_parser=RouterOutputParser())
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [10]:
chain = MultiRouteChain(router_chain=router_chain, destination_chains=destination_chains,default_chain=default_chain, verbose=True)

In [15]:
result = chain.invoke("What is the derivative of x dx")



> Entering new MultiRouteChain chain...


/Users/prasadk/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Calculus Teacher: {'input': 'What is the derivative of x dx'}
> Finished chain.


In [16]:
print (result["text"])

Hello, I am Godfrey, your Calculus teacher.

The derivative of $x^ndx$ is $n x^(n-1)$.

In this case, $n = 1$, so the derivative of $x dx$ is $1 x^(1-1) = 1x^0 = 1$.

Another way to think about it is that the derivative of $x^n$ is the power of $x$ multiplied by the coefficient of $x^n$.

In this case, the power of $x$ is $1$ and the coefficient of $x^1$ is $1$, so the derivative of $x dx$ is $1 \cdot 1 = 1$.
